# Numerical Representations
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/num.ipynb)

In [ ]:
class Num n where
    
    -- Increment given number by one
    inc :: n -> n
    
    -- Decrement given number by one
    dec :: n -> n


class Num n => Add n where

    -- Add two numbers together
    add :: n -> n -> n

## Dense Binary Numbers
Let $N \in \mathbb{N}$ have a binary represetation $b_0 \dots b_{m-1}$ with digits $b_i \in \{0, 1\}$, i.e. $N = \sum_{i=0}^{m-1} b_i 2^i$. Then the *dense representation* explicitly stores each digit $b_i$.

Since there can be at most logarithmuc nuber of 0s and 1s, both `inc` and `dec` run in $O(log(n))$.

In [ ]:
{-# LANGUAGE FlexibleInstances #-}

data Digit = Zero | One

-- | Dense binary numbers as a list of 'Digit's in increasing order of significance
type DenseNat = [Digit]

instance Num DenseNat where
    
    inc [] = [One]
    inc (Zero:ds) = One : ds
    inc (One:ds) = Zero : inc ds -- carry
    
    dec [One] = []
    dec (One:ds) = Zero : ds
    dec (Zero:ds) = One : dec ds -- borrow
    
instance Add DenseNat where
    add ds [] = ds
    add [] ds = ds
    add (d:ds1) (Zero:ds2) = d : add ds1 ds2
    add (Zero:ds1) (d:ds2) = d : add ds1 ds2
    add (One:ds1) (One:ds2) = Zero : inc (add ds1 ds2) -- carry

## Sparse Binary Numbers
Contrary to the *dense representation* which stores zeros, in the *sparse representation* zeros are omitted.

Sparse encoding, however, must still preserve complete information (must be representative). Therefore in sparse representation one stores ranks or weights of the non-zero digits $b_i$ rather than the digits themself - e.g. for binary numbers one'd store just those weights $w_i = 2^i$ for which $b_i = 1$.

In [ ]:
{-# LANGUAGE FlexibleInstances #-}

-- | Sparse binary numbers as an increasing list of weights, each a power of 2
type SparseNat = [Int]

carry :: Int -> SparseNat -> SparseNat
carry w [] = [w]
carry w ws @ (w':ws') = if w < w' then w : ws else carry (2*w) ws'

borrow :: Int -> SparseNat -> SparseNat
borrow w ws @ (w':ws') = if w == w' then ws' else w : borrow (2*w) ws

instance Num SparseNat where
    
    inc = carry 1
    
    dec = borrow 1
    
instance Add SparseNat where
    add ws [] = ws
    add [] ws = ws
    add m @ (w1:ws1) n @ (w2:ws2) = case compare w1 w2 of
        LT -> w1 : add ws1 n
        GT -> w2 : add m ws2
        EQ -> carry (2*w1) (add ws1 ws2)

## Zeroless Binary Numbers
*Zeroless binary numbers* are similar to ordinary binary numbers but are constructed from 1s and 2s. There's no zero in this representation but the weight of the $i$-th digit is still $2^i$.

For example the decimal number 16 can be written as $\text{2111}$ instead of $\text{00001}$ (note: in our binary represendation LSB is digit $b_0$).

### Benefits
The main benefit of this representation is that all digits are non-zero. This can be exploited by data structures using a dense representation to efficiently store and access data at the first position.

In [ ]:
-- | Binary digit for which Zero -> One and One -> Two
data Digit = One | Two
    deriving (Show, Eq)

-- | Dense zeroless representation as a list of non-zero digits in increasing order of weigths 2^i
type ZerolessNat = [Digit]

instance Num ZerolessNat where

    inc [] = [One]
    inc (One:ds) = Two : ds
    inc (Two:ds) = One : inc ds
    
    dec [One] = []
    dec (Two:ds) = One : ds
    dec (One:ds) = Two : dec ds
    
instance Add ZerolessNat where
    -- | Note: Compared to 'DenseNat', here carries involve either 1s or 2s.
    add ds [] = ds
    add [] ds = ds
    add (One:ds1) (One:ds2) = Two : Ghci2.add ds1 ds2
    add (One:ds1) (Two:ds2) = One : inc (Ghci2.add ds1 ds2)
    add (Two:ds1) (d:ds2) = d : inc (Ghci2.add ds1 ds2)

## Redundant Binary Numbers

In the implementation of `LazyNat` presented below, we make a case for a [*redundant representation*](https://en.wikipedia.org/wiki/Redundant_binary_representation) of binary numbers.

The motivation behind `LazyNat` is to amortize operations to $O(1)$ via lazy evaluation. As the book demonstrates, there is a problem with a straightforward modification of `DenseNat` - although both `inc` and `dec` can be made amortized $O(1)$ independently but this breaks when used in combination.

Informally, one operation makes one of the digits "dangerous" (expensive) while the other operation does the exact opposite. The solution to this probelm is to introduce another redundant digit that is "safe" for both operations.

In [ ]:
-- | Redundant representation of binary digits where
-- |  - 'Zero' is dangerous for the 'inc' operation
-- |  - 'One' is safe for both operations
-- |  - 'Two' is dangerous for the 'dec' operation
data Digit = Zero | One | Two

-- | Lazy, dense and redundant representation of binary numbers
type LazyNat = [Digit]

instance Num LazyNat where

    -- | Note: Runs in O(1) amortized time.
    inc [] = [One]
    inc (Zero:ds) = One : ds
    inc (One:ds) = Two : ds
    inc (Two:ds) = One : inc ds
    
    -- | Note: Runs in O(1) amortized time.
    dec [One] = []
    dec (One:ds) = Zero : ds
    dec (Two:ds) = One : ds
    dec (Zero:ds) = One : dec ds
    
instance Add LazyNat where
    add ds [] = ds
    add [] ds = ds
    add (d:ds1) (Zero:ds2) = d : add ds1 ds2
    add (Zero:ds1) (d:ds2) = d : add ds1 ds2
    add (One:ds1) (One:ds2) = Two : add ds1 ds2
    add (d:ds1) (Two:ds2) = d : inc (add ds1 ds2)
    add (Two:ds1) (d:ds2) = d : inc (add ds1 ds2)

## Segmented Binary Numbers
The representation of *segmented binary numbers* `SegmentedNat` is similar to the ordinary `DenseNat` but with digits grouped into blocks of either 0s or 1s.

The benefit of this segmentation is that carries and borrows do not cascade and thus can be carried over whole blocks in constant time (all at once). Due to this property, `inc` and `dec` operations run in $O(1)$ worst case time.

Note that although this representation provides efficinent operations, usually it is not directly possible to translate this numerical representation to more complicated structures (e.g. to trees).

In [ ]:
-- | Segment certain number of either Os or 1s
data DigitBlock = Zeros Int | Ones Int

-- | Segmented (dense) representation of binary numbers
type SegmentedNat = [DigitBlock]

-- | Helper that merges adjecent blocks of 0s, discards empty blocks and removes trailing 0s.
-- |
-- | Note: Due to the 'DigitBlock' representation this function runs in O(1) worst case time.
zeros :: Int -> SegmentedNat -> SegmentedNat
zeros _ [] = []
zeros 0 bs = bs
zeros i ((Zeros j):bs) = Zeros (i + j) : bs
zeros i bs = Zeros i : bs

-- | Helper that merges adjecent blocks of 1s and discards empty blocks.
-- |
-- | Note: Due to the 'DigitBlock' representation this function runs in O(1) worst case time.
ones :: Int -> SegmentedNat -> SegmentedNat
ones 0 bs = bs
ones i ((Ones j):bs) = Ones (i + j) : bs
ones i bs = Ones i : bs

instance Num SegmentedNat where

    -- | Note: Runs in O(1) worst case time based on the analysis of 'ones' and 'zeros'.
    inc [] = [Ones 1]
    inc ((Zeros i):bs) = ones 1 $ zeros (i - 1) bs
    inc ((Ones i):bs) = Zeros i : inc bs
    
    -- | Note: Runs in O(1) worst case time based on the analysis of 'ones' and 'zeros'.
    dec ((Ones i):bs) = zeros 1 $ ones (i - 1) bs
    dec ((Zeros i):bs) = Ones i : dec bs
    
instance Add SegmentedNat where
    -- | Note: This implementation expects properly segmented inputs. If this cannot be assumed,
    -- |       one could use for instance `zeros (j - i) ds2` in `add ds1 (Zeros (j - i) : ds2)`.
    add bs [] = bs
    add [] bs = bs
    add ((Zeros i):ds1) ((Zeros j):ds2) =
        if i < j then zeros i $ add ds1 (Zeros (j - i) : ds2)
        else zeros j $ add (Zeros (i - j) : ds1) ds2
    add ((Ones i):ds1) ((Zeros j):ds2) =
        if i < j then ones i $ add ds1 (Zeros (j - i) : ds2)
        else ones i $ add (Ones (i - j) : ds1) ds2
    add ((Zeros i):ds1) ((Ones j):ds2) =
        if i < j then ones i $ add ds1 (Ones (j - i) : ds2)
        else ones i $ add (Zeros (i - j) : ds1) ds2
    add ((Ones i):ds1) ((Ones j):ds2) =
        if i < j then zeros i $ inc (add ds1 (Ones (j - i) : ds2))
        else zeros j $ inc (add (Ones (i - j) : ds1) ds2)

## Redundant Segmented Binary Numbers
`RedundantSegmentedNat` is a more practical variant of `SegmentedNat` which combines *redundant* and *segmented representation*.

### Representation

The representation is similar to `LazyNat` in that there are two "dangerous" digits (`Zero` and `Two`) but the "safe" digit is actually a whole block of `Ones` - just as in `SegmentedNat`.

### Operations

One can consider a `Two` as a carry in progress. Then to prevent a cascade of carries, one has to guarantee that there are no more than one `Two` in a row. In regular expressions, this invariant translates to either of these two:
 - $(\text{0}\:|\:\text{1}\:|\:\text{01}^*\text{2})^*$
 - $(\text{0}^*\text{1}\:|\:\text{0}^+\text{1}^*\text{2}^*)^*$ (without trailing zeros)

I.e. the last non-one digit before each `Two` is a `Zero`.

Because the first digit is never a `Two`, `inc` can run in $O(1)$ worst case time - by simply incrementing the first digit and then fixing the invariant. `dec`, on the other hand, may take $O(log(n))$ time in the worst case.

### Generalization (*recursive slowdown*)

This numerical representation can be viewed as a template for data structures composed of a sequence of levels where each level can be classified as
 - *green* corresponds to `Zero` in the above interpretation
 - *yellow* corresponds to `One` in the above interpretation 
 - *red* corresponds to `Two` in the above interpretation

> An operation may degrade the color of the first level from *green* to *yellow* or from *yellow* to *red* but never from *green* to *red*. [...] The invariant is that the last non-yellow level before a red level is always green. [...] Consecutive *yellow* levels are grouped into a block to support efficient access to the first non-yellow level.

For instance a *segmented binomial heap* that supports `insert` in $O(1)$ worst case time can be represented as follows:
```haskell
data Tree a = Node a [Tree a]

data Digit a = Zero | Ones [Tree a] | Two (Tree a) (Tree a)

type SegmentedBinomialHeap a = [Digit a]
```

Note: The invariant is restored after each `merge` by eliminating all `Two`s.

In [ ]:
-- | Semi-segmented representation of redundant binary numbers
data Digits = Zero | Ones Int | Two

type RedundantSegmentedNat = [Digits]

-- | Helper to merge adjacent blocks of 1s and delete empty blocks.
-- |
-- | Note: This function runs  in O(1) worst case time.
ones :: Int -> RedundantSegmentedNat -> RedundantSegmentedNat
ones 0 ds = ds
ones i ((Ones j):ds) = Ones (i + j) : ds
ones i ds = Ones i : ds

-- | Increment given number by blindly incrementing the first digit.
-- |
-- | Note: Runs in O(1) worst case time.
simpleinc :: RedundantSegmentedNat -> RedundantSegmentedNat
simpleinc [] = [Ones 1]
simpleinc (Zero:ds) = ones 1 ds
simpleinc ((Ones i):ds) = Two : ones (i - 1) ds

-- | Restore the invariant by fixing leading 'Two' and the first non-one digit in a sequence 'ds'.
-- |
-- | Note: Runs in O(1) worst case time.
fixup :: RedundantSegmentedNat -> RedundantSegmentedNat
fixup (Two:ds) = Zero : simpleinc ds
fixup ((Ones i):Two:ds) = Ones i : Zero : simpleinc ds
fixup ds = ds

instance Num RedundantSegmentedNat where
    
    -- | Increment given number in O(1) worst case time.
    inc = fixup . simpleinc
    
    -- | Decrement given number in O(log(n)) worst case time.
    -- |
    -- | This implementation is a combination of 'dec' for 'LazyNat' and 'SegmentedNat'.
    -- | The complexity comes from the fact that 0s (and 2s) are dense and not segmented.
    dec ((Ones i):ds) = Zero : ones (i - 1) ds
    dec (Two:ds) = ones 1 ds
    dec (Zero:ds) = ones 1 $ dec ds
    
-- TODO: `instance Add RedundantSegmentedNat`

As mentioned above, `dec` may unfortunately take up to $O(log(n))$ time in the worst case.

However, the complexity of `dec` can be improved to $O(1)$ by extending the redundant representation to digits 0, 1, 2, 3 and 4 where
 - `Zero` and `Four` are *red* (i.e. borrow and carry in progress)
 - `Ones` and `Threes` are *yellow* (represented as segments)
 - and `Two` is *green*

For the `RedundantSegmentedNat` above we maintained the invariant that there is no more than one *red* digit in a row (i.e. the first non-one digit had to be `Zero` - the *green* digit). Similarly, here we must ensure that the first non-one, non-three is `Two` (which is now *green*).

TODO: Check whether the implementation below is correct. Most importantly the fixup functions. Also implement `Add` instance for `SegmentedRedundantNat`.

In [ ]:
-- | Segmented representation of redundant binary numbers
data Digits = Zero | Ones Int | Two | Threes Int | Four

type SegmentedRedundantNat = [Digits]

-- | Helper to merge adjacent blocks of 1s and delete empty blocks.
-- |
-- | Note: This function runs  in O(1) worst case time.
ones :: Int -> SegmentedRedundantNat -> SegmentedRedundantNat
ones 0 ds = ds
ones i ((Ones j):ds) = Ones (i + j) : ds
ones i ds = Ones i : ds

-- | Helper to merge adjacent blocks of 3s and delete empty blocks.
-- |
-- | Note: This function runs  in O(1) worst case time.
threes :: Int -> SegmentedRedundantNat -> SegmentedRedundantNat
threes 0 ds = ds
threes i ((Threes j):ds) = Threes (i + j) : ds
threes i ds = Threes i : ds

-- | Increment given number by blindly incrementing the first digit.
-- |
-- | Note: Runs in O(1) worst case time.
simpleinc :: SegmentedRedundantNat -> SegmentedRedundantNat
simpleinc [] = [Ones 1]
simpleinc (Zero:ds) = ones 1 ds
simpleinc (Two:ds) = threes 1 ds
simpleinc ((Ones i):ds) = Two : ones (i - 1) ds
simpleinc ((Threes i):ds) = Four : threes (i - 1) ds

-- | TODO: check if 'Two' is correct
-- |
-- | Restore the invariant after 'simpleinc' by fixing leading 'Four' and
-- | the first non-three digit in a sequence 'ds'.
-- |
-- | Note: Runs in O(1) worst case time.
fixupinc :: SegmentedRedundantNat -> SegmentedRedundantNat
fixupinc (Four:ds) = Two : simpleinc ds
fixupinc ((Threes i):Four:ds) = Threes i : Two : simpleinc ds
fixupinc ds = ds

-- | Decrement given number by blindly decrementing the first digit.
-- |
-- | Note: Runs in O(1) worst case time.
simpledec :: SegmentedRedundantNat -> SegmentedRedundantNat
simpledec (Two:ds) = ones 1 ds
simpledec (Four:ds) = threes 1 ds
simpledec ((Ones i):ds) = Zero : ones (i - 1) ds
simpledec ((Threes i):ds) = Two : threes (i - 1) ds

-- | TODO: check if 'Two' is correct
-- |
-- | Restore the invariant after 'simpledec' by fixing leading 'Zero' and
-- | the first non-one digit in a sequence 'ds'.
-- |
-- | Note: Runs in O(1) worst case time.
fixupdec :: SegmentedRedundantNat -> SegmentedRedundantNat
fixupdec (Zero:ds) = Two : simpledec ds
fixupdec ((Ones i):Zero:ds) = Ones i : Two : simpledec ds
fixupdec ds = ds

instance Num SegmentedRedundantNat where
    
    -- | Increment given number in O(1) worst case time.
    inc = fixupinc . simpleinc
    
    -- | Decrement given number in O(1) worst case time.
    dec = fixupdec . simpledec
    
-- TODO: `instance Add SegmentedRedundantNat`

## Skew Binary Numbers
*Skew representation* of binary numbers defines weights $w_i = 2^{i + 1} - 1$ instead of typical $2^i$ and the set of digits is $D_i = \{\text{0}, \text{1}, \text{2}\}$.

### Representation
This representation is redundant but can be made unique (in *canonical form*) if one adds the restriction that only the lowest non-zero digit may be two.
> Every natural number has a unique skew binary canonical form.

Since `inc` scans for the lowest non-zero digit (see operations below), the natural choice for representation is *sparse*. Using *dense* would require more than $O(1)$ steps which defeats the purpose.

### Operations
Observation: $1 + 2(2^{i + 1} - 1) = 2^{i + 2} - 1$. This implies
 1. `inc` a number that doen't contain 2 => just increment the lowest digit: 0 -> 1 or 1 -> 2
 1. `inc` a number with the lowest non-zero digit being 2 => reset it to 0 and increment the next digit

### Benefits
 - Similarly to *lazy* and *segmented* representations, both `inc` and `dec` run in $O(1)$ worst case time in the *skew representation* (instead of $O(log(n))$)
 - The implementation is ususally simpler and faster in practice

In [ ]:
-- | Sparse representation of skew binary numbers as a list of weights.
-- |  - weight are stored in increasing order
-- |  - only the first two (smallest) weight may be the same - indicating that the lowest non-zero digit is two
type SkewNat = [Int]

instance Num SkewNat where
    -- | Increment given number in O(1) worst case time.
    inc ws @ (w1:w2:rs) = if w1 == w2 then (1 + w1 + w2):rs else 1:ws
    inc ws = 1:ws
    
    -- | Decrement given number in O(1) worst case time.
    -- |
    -- | Note: If 'w = 2^(k + 1) - 1' then '(w `div` 2) = 2^k - 1'.
    dec (1:ws) = ws
    dec (w:ws) = (w `div` 2) : (w `div` 2) : ws